In [15]:
import numpy as np
import pandas as pd
import datetime as dt
import altair as alt
alt.data_transformers.enable("json")

DataTransformerRegistry.enable('json')

# Open data

Radiosonde synoptic winds

In [16]:
df1 = pd.read_parquet("../sail/synoptic_winds_500_local.parquet").assign(pressure = 500)
df2 = pd.read_parquet("../sail/synoptic_winds_700_local.parquet").assign(pressure = 700)
df1.time = df1.time.apply(lambda dt: dt.replace(minute=0, second=0))
df2.time = df2.time.apply(lambda dt: dt.replace(minute=0, second=0))
synopticwinds_df = pd.concat([df1, df2])
synopticwinds_df = synopticwinds_df.groupby(['time', 'pressure']).mean().reset_index()

In [17]:
alt.Chart(
    synopticwinds_df
).mark_bar().encode(
    alt.X("wspd:Q").bin(True, maxbins=30),
    alt.Y("count():Q")
)

alt.Chart(...)

Doppler Lidar (aggregated) data

In [18]:
dl_df = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/sail_processed/gucdlrhiM1.b1/").set_index([
    'z_binned',	'x_offset',	'scan_time'
])['streamwise_velocity'].reset_index()

dl_df = dl_df[dl_df.scan_time >= "2023-02-09 0000"]


# Plot vertical profiles over a day

In [5]:
src = dl_df[dl_df.scan_time.dt.date == dt.date(2023,4,16)].query("x_offset == -250")
src = src[src.z_binned < 800]

In [6]:
alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("minutes(scan_time):O").scale(scheme='turbo'),
    alt.Facet("hours(scan_time):T", columns=6)
).properties(width = 125, height = 125, title = f"Along-valley wind profiles on {str(src.scan_time.dt.date.iloc[0])}")

alt.Chart(...)

In [7]:
src = dl_df[dl_df.scan_time.dt.date == dt.date(2023,4,16)].query("x_offset == -250")
src['hour'] = src.scan_time.dt.hour
src['minute'] = src.scan_time.dt.minute
src = src[src.z_binned < 800]
alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("minute:O").scale(scheme='turbo'),
    alt.Facet("hour", columns=6)
).properties(width = 125, height = 125, title = f"Along-valley wind profiles on {str(src.scan_time.dt.date.iloc[0])}")

alt.Chart(...)

# Plot daytime/nighttime-averaged wind speed profiles over the season

In [8]:
src = dl_df.query("x_offset == -250").dropna()
def categorize_daytime(hr):
    if hr in [9,10,11,12,13,14,15]:
        return 'day'
    elif hr in [19,20,21,22,23,0,1,2,3,4,5]:
        return 'night'
    else:
        return np.nan
src.loc[:, ['daytime_category']] = src.scan_time.dt.hour.apply(categorize_daytime)
src = src.dropna()
src['date'] = src['scan_time'].dt.date
src = src.groupby(['daytime_category', 'date', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

In [9]:
# Add upper bounds to z and to time for a hovmoller diagram
upper_bound_to_lower_bound = dict(zip(
    sorted(src.z_binned.unique()),
    [0] + sorted(src.z_binned.unique())
))
src['z_high'] = src['z_binned']
src['z_low'] = src['z_high'].apply(upper_bound_to_lower_bound.get)
src['date_high'] = src['date'] + dt.timedelta(hours=24)

In [10]:
night_chart = alt.Chart(
    src.query("daytime_category == 'night'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Night time winds (1900 - 0500)"
)

day_chart = alt.Chart(
    src.query("daytime_category == 'day'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Day time winds (0900 - 1500)"
)

(night_chart & day_chart).configure_legend(gradientThickness=20, gradientLength = 150)

alt.VConcatChart(...)

# Identify strong/weak synoptics based on 500 mb wind speeds (from radiosonde data)

In [11]:
synoptic_windspeed_daily = synopticwinds_df.groupby(
    [synopticwinds_df.time.dt.date, 'pressure']
)[['wspd']].mean().reset_index().query("pressure == 500")
low_synoptic_dates = synoptic_windspeed_daily.query("wspd < 15").time

Plot the same as above, excluding strong synoptic days

In [12]:
src[src.date.isin(low_synoptic_dates)].query("daytime_category == 'night'").query("z_binned < 600")

,daytime_category,date,z_binned,mean,median,std,z_high,z_low,date_high


In [13]:

alt.Chart(
    src[src.date.isin(low_synoptic_dates)].query("daytime_category == 'night'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True)
).properties(width = 600, height = 200)

alt.Chart(...)

In [43]:

alt.Chart(
    src[src.date.isin(low_synoptic_dates)].query("daytime_category == 'day'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True)
).properties(width = 600, height = 200)

alt.Chart(...)

# Normalize wind speeds by synoptic wind speeds

In [44]:
synoptic_windspeed_daily.head()

,time,pressure,wspd
0,2023-02-01,500,10.261045
2,2023-02-02,500,8.197502
4,2023-02-03,500,10.127381
6,2023-02-04,500,20.670246
8,2023-02-05,500,30.192680


In [45]:
src_normalized = src.merge(
    synoptic_windspeed_daily,
    left_on='date',
    right_on = 'time',
    how='left'
)
src_normalized['median_normalized'] = src_normalized['median'] / src_normalized['wspd']

In [46]:
night_chart = alt.Chart(
    src_normalized.query("daytime_category == 'night'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median_normalized:Q").scale(scheme='purpleorange', domain=[-0.5,0.5], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Night time winds (1900 - 0500)"
)

day_chart = alt.Chart(
    src_normalized.query("daytime_category == 'day'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median_normalized:Q").scale(scheme='purpleorange', domain=[-0.5,0.5], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Day time winds (0900 - 1500)"
)

(night_chart & day_chart).configure_legend(gradientThickness=20, gradientLength = 150)

alt.VConcatChart(...)

# Plot wind speed hovmoller diagram (using speeds 25 - 175m)

In [47]:
src = dl_df.query("x_offset == -250").dropna()
src = src[
    (src.z_binned >= 25) &
    (src.z_binned <= 175)
]
src = src.dropna()
src['date'] = src['scan_time'].dt.date
src['hour'] = src['scan_time'].dt.hour
src = src.groupby(['date', 'hour'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

# Add upper bounds to z and to time for a hovmoller diagram
src['date_high'] = src['date'] + dt.timedelta(hours=24)
src['hour_high'] = src['hour'] + 1

src.head()

,date,hour,mean,median,std,date_high,hour_high
0,2023-02-10,16,-0.339759,-0.418169,2.324586,2023-02-11,17
1,2023-02-10,17,2.599068,2.536814,6.665049,2023-02-11,18
2,2023-02-10,18,0.800708,0.688943,6.396798,2023-02-11,19
3,2023-02-10,19,0.025642,0.736871,4.917403,2023-02-11,20
4,2023-02-10,20,-5.660494,-3.378579,7.471944,2023-02-11,21


In [48]:
alt.Chart(
    src
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("hour:Q").title("hour of day"),
    alt.Y2("hour_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Average wind speed, 25 - 175m"
).display(renderer='svg')

alt.Chart(...)

# Plot monthly-averaged vertical profiles

In [50]:
low_synoptic_dates

0    2023-02-01
2    2023-02-02
4    2023-02-03
Name: time, dtype: object

In [92]:
dl_df.query("z_binned == 15.0").query("x_offset == -250").groupby(dl_df.scan_time.dt.month).count()

,z_binned,x_offset,scan_time,mean,median,std,count
scan_time,,,,,,,
2,1758,1758,1758,1735,1735,1641,1758
3,2972,2972,2972,2906,2906,2725,2972
4,2880,2880,2880,2871,2871,2836,2880
5,2976,2976,2976,2976,2976,2961,2976
6,1485,1485,1485,1485,1485,1485,1485


In [93]:
src

,month,hour,z_binned,mean,median,std
0,2,0,5.0,0.972627,0.232884,1.942128
1,2,0,15.0,1.022988,1.497172,0.580861
2,2,0,25.0,1.129894,1.679058,0.451016
3,2,0,35.0,1.221973,1.490303,0.593153
4,2,0,45.0,1.076573,1.895230,0.568781
...,...,...,...,...,...,...
5754,6,23,1775.0,2.673607,3.296719,1.960636
5755,6,23,1825.0,3.552808,4.454063,1.674921
5756,6,23,1875.0,2.785776,3.860508,2.092473
5757,6,23,1925.0,4.066625,5.710420,2.959328


In [84]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour

src = src.groupby(['month', 'hour', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()


alt.Chart(
    src.query("z_binned <= 200").query("month < 6")
).mark_line().encode(
    alt.X("mean:Q").sort('-y'),
    alt.Y("z_binned:Q"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Row("hour") 
).properties(width = 125, height = 125) | alt.Chart(
    src.query("z_binned <= 200").query("month < 6")
).mark_line().encode(
    alt.X("median:Q").sort('-y'),
    alt.Y("z_binned:Q"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Row("hour") 
).properties(width = 125, height = 125)

alt.HConcatChart(...)

In [77]:
alt.layer(
    alt.Chart().mark_rule().encode(x=alt.datum(0)),
    alt.Chart().mark_line().encode(
        alt.X("mean:Q").sort('-y'),
        alt.Y("z_binned:Q"),
        alt.Color("month:O").scale(scheme='turbo'),
        
    ),
    data=src.query("z_binned <= 200").query("month < 6")
).properties(width = 125, height = 125).facet("hour:O", columns=6)

alt.FacetChart(...)

In [24]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour
src['hour_group'] = pd.cut(
    src['hour'], 
    [-1,2.5, 5.5, 8.5, 11.5, 14.5, 17.5, 20.5, 23.5],
    labels=['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']
)

src = src.groupby(['month', 'hour_group', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

alt.Chart(
    src.query("z_binned <= 625").query("month < 6")
).mark_line().encode(
    alt.X("mean:Q").sort('-y').title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Facet("hour_group", columns=4).title("hours of day")
).properties(
    width = 125, 
    height = 125,
    title='Mean vertical profiles of valley wind for different parts of the day'
).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_57291/3937551297.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  src = src.groupby(['month', 'hour_group', 'z_binned'])[


alt.Chart(...)

In [106]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour
src['hour_group'] = pd.cut(
    src['hour'], 
    [-1,2.5, 5.5, 8.5, 11.5, 14.5, 17.5, 20.5, 23.5],
    labels=['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']
)

src = src.groupby(['month', 'hour_group', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

src = src.query("z_binned <= 625").query("month < 6")
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']

# src = src[src.month.isin([2,5])]

error_bands_chart = alt.Chart(
    src
).mark_errorband().encode(
    alt.X("lower_bound:Q"),
    alt.X2("upper_bound:Q"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='rainbow')
)

lines_chart = alt.Chart(
    src
).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='turbo')
)
# title='Median vertical profiles of valley wind for different parts of the day'

(
    # error_bands_chart + 
 lines_chart).properties(
    width = 125, height = 125, title="hours of day"
).facet("hour_group", columns=4).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_57291/2401788342.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  src = src.groupby(['month', 'hour_group', 'z_binned'])[


alt.FacetChart(...)

In [ ]:


lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
).properties(width = 150, height = 150)

In [21]:
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date.isin([
    dt.date(2023, 4, 16),
    dt.date(2023, 6, 9)
])]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)

In [22]:
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']

In [23]:
error_bands_chart = alt.Chart(src).mark_errorband().encode(
    alt.X("lower_bound:Q").title("Downvalley velocity (m/s)"),
    alt.X2("upper_bound:Q").title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
).properties(width = 150, height = 150)

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
).properties(width = 150, height = 150)


(error_bands_chart + lines_chart).facet(
    row = alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    column = alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
)

alt.FacetChart(...)

In [42]:
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date.isin([
    dt.date(2023, 4, 16),
    # dt.date(2023, 6, 9)
])]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby(['scan_time', 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['minute'] = src['scan_time'].dt.minute
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']

error_bands_chart = alt.Chart(src).mark_errorband().encode(
    alt.X("lower_bound:Q").title("Downvalley velocity (m/s)").scale(domain=[-15,15]),
    alt.X2("upper_bound:Q").title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("minute:N"),
).properties(width = 150, height = 150)

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley velocity (m/s)").scale(domain=[-15,15]),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("minute:N"),
).properties(width = 150, height = 150)


(error_bands_chart + lines_chart).facet(
    "hours(scan_time):T", columns=6
)

alt.FacetChart(...)

In [26]:
xxxx = src[src.hour.isin([2, 12])]

xxxx.time_category = xxxx.time_category.replace({
    'Day (0800-1400)':      'Day',
    'Morning (0000-0600)':  'Night',
})

error_bands_chart = alt.Chart(xxxx).mark_errorband().encode(
    alt.X("lower_bound:Q").title("Downvalley velocity (m/s)"),
    alt.X2("upper_bound:Q").title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("date_str:N"),
).properties(width = 125, height = 125)

lines_chart = alt.Chart(xxxx).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("date_str:N"),
).properties(width = 125, height = 125)


(error_bands_chart + lines_chart).facet(
    row = alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    column = alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_20496/1746103850.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  xxxx.time_category = xxxx.time_category.replace({
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_20496/1746103850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xxxx.time_category = xxxx.time_category.replace({


alt.FacetChart(...)